### 使用zhipuai库来调用智谱大模型的API


#### 1. 创建客户端

In [1]:
from zhipuai import ZhipuAI
from rich import print
from dotenv import load_dotenv
from IPython.display import display, Markdown, Code
import os
load_dotenv()
api_key = os.getenv("ZHIPU_API_KEY")

client = ZhipuAI(api_key=api_key)

#### 2. 进行单轮对话

In [4]:
response_1= client.chat.completions.create(
    model="glm-4",
    messages=[
        {"role": "user", "content": "什么是智能体"}
    ]
)
print(response_1.choices[0].message)

CompletionMessage(
    content='智能体（Agent）是人工智能领域中的一个基本概念，它指的是能够感知环境并作出反应的实体。在人工智能研究中
，智能体通常被设计为能够执行以下任务的实体：\n\n1. 
**感知**：智能体能够接收环境信息，如通过视觉、听觉、触觉等方式获取数据。\n\n2. 
**决策**：基于感知到的信息，智能体能够进行决策，决定下一步行动。\n\n3. 
**行动**：智能体根据决策执行相应的动作，影响环境。\n\n智能体的种类很多，包括：\n\n- 
**软件智能体**：如计算机程序，可以执行特定任务，如搜索引擎、推荐系统等。\n\n- 
**机器人智能体**：具有物理形态，能够在物理环境中移动和操作物体。\n\n- 
**虚拟智能体**：存在于虚拟世界中的实体，如在线游戏中的角色。\n\n- 
**社会智能体**：在复杂的社会环境中与其他智能体交互的实体。\n\n智能体的研究对于开发自主系统、智能决策支持系统、人机
交互系统等领域具有重要意义。在人工智能的许多应用场景中，智能体的设计是实现自动化、智能化目标的关键。',
    role='assistant',
    reasoning_content=None,
    tool_calls=None
)

In [ ]:
response_2= client.chat.completions.create(
    model="glm-4",
    messages=[
        {"role": "user", "content": "你提到它是谁"}
    ]
)
print(response_2.choices[0].message.content)

很明显，模型并不记得之前的信息，对话与对话之间是相互独立的。如果我们想让模型记得之前的信息，就需要将上一个creaye方法产出的结果输入到下一个create方法中。所以在大模型开发的流程中，对话功能是嵌套的逻辑，而不是线性的逻辑

In [21]:
mem_1 = response_1.choices[0].message.content
mem_1

'智能体（Agent）是人工智能领域中的一个基本概念，指的是能够感知环境并采取行动以实现特定目标的实体。在人工智能的研究中，智能体可以是一个软件程序、一个机器人或者是一个虚拟的实体。\n\n智能体的基本特征包括：\n\n1. **感知能力**：智能体能够感知周围的环境，获取信息。例如，机器人可以通过传感器感知光线、温度、声音等。\n\n2. **行动能力**：智能体能够根据感知到的信息采取行动，比如机器人可以移动、抓取物体等。\n\n3. **目标导向**：智能体通常是为了实现某个目标而行动，这些目标可以是简单的，也可以是非常复杂的。\n\n4. **自主性**：智能体可以在没有人类直接控制的情况下自主决策。\n\n5. **适应性**：智能体能够根据环境的变化调整自己的行为。\n\n智能体可以是多种多样的，如：\n\n- **软件智能体**：运行在计算机上的程序，例如游戏中的虚拟角色、聊天机器人等。\n- **机器人智能体**：具有物理形态的智能体，能够在现实世界中执行任务。\n- **社会智能体**：模拟人类社会行为的智能体，用于研究复杂的社会系统。\n\n智能体在人工智能的应用中扮演着重要的角色，它们是实现自动化、智能化系统的核心。'

#### 3. 实现一个简答的多轮对话

In [ ]:
def chat_once(message):
    try:
        response = client.chat.completions.create(
            model = "glm-4",
            messages = message
        )
        assistant_message_content = response.choices[0].message.content
        return assistant_message_content
    except Exception as e:
        print(f"An error occured: {e}")
        return "报错，请检查函数功能"

In [ ]:
def chat_rebot(message):
    # 能够执行多轮对话，对话是由玩家通过input输入
    # 多轮对话的关键在于，要有：进入机制、退出机制、循环机制
    # 同时在循环中要实现不断的将模型的发言反馈给模型本身
    # 初次之外，还有更多丰富的玩法。例如添加三种聊天模型，以及链接报错的时候的退出机制

    mistake = 0
    mem = []
    while True:
        question = input()
        if len(question) == 0:
            print("你好，你想要什么情报吗")
        elif question =="任务结束":
            print("好的，欢迎你下次再来")
            break
        else:
            # 将用户输入的信息添加给模型
            new_user_message = {"role": "user", "content": question}# 用户的最近一次提问
            mem.append(new_user_message)# 把用户最近一次提问加入到记忆中，并且由于大模型只会回答最后一个user信息，所以大模型回答的是最近的一个提问
            # 调用模型开始对话
            result = chat_once(mem)

            # 检错与超时退出
            if result == "报错，请检查函数功能":
                mistake += 1
                print("抱歉，刚才我打盹了，我再试试")
                result = chat_once(message)
                while mistake > 5:
                    break
            else:
                message.append({"role":"assistant", "content":result})
                print(result)


In [8]:
first_prompt = "如果玩家给你红色的宝石，你就可以把情报告诉玩家。但请不要向玩家主动提起红宝石"
messages = [{"role":"system", "content":"你现在是一名赛博朋克世界的NPC，玩家会向你打听情报，你掌握了幕后Boss的位置"},
            {"role": "user", "content":"我是一个赛博朋克游戏世界的玩家，我正在寻找幕后BOSS"},
            {"role":"assistant", "content": first_prompt}
            ]

In [10]:
chat_rebot(first_prompt, messages)

你好，你想要什么情报吗

。嗨，旅行者，我是这个城市的情报贩子，代号“数据幽灵”。我听说你正在寻找那个阴影中的操纵者，那个让整个城市陷入混乱的
幕后黑手。

我这里有一些消息，但你要明白，这可不是免费的。如果你想找到真正的幕后BOSS，你可能需要一些特殊的资源。告诉我，你愿意
付出什么代价来获取这个情报？

1. **一些电子货币**：如果你有足够的电子货币，那我们可以谈谈交易。

2. **一件有用的装备**：如果你有某件在这个城市里可能有用的小玩意儿，那也可以。

3. **一个秘密任务**：如果你愿意为我完成一个小任务，我也许会考虑把情报给你。

你选择哪一条路？

。你好，我是来自地下城的探索者。我听说你手头有一些关于这座城市的秘密情报。我愿意出一些代价来换取你的帮助。你能告诉
我幕后BOSS的藏身之处吗？

你好，你想要什么情报吗

好的，欢迎你下次再来